# MAGIC Gamma Telescope - TPOT Classification Study

The below gives information about the data set:

The data are MC generated (see below) to simulate registration of high energy gamma particles in a ground-based atmospheric Cherenkov gamma telescope using the imaging technique. Cherenkov gamma telescope observes high energy gamma rays, taking advantage of the radiation emitted by charged particles produced inside the electromagnetic showers initiated by the gammas, and developing in the atmosphere. This Cherenkov radiation (of visible to UV wavelengths) leaks through the atmosphere and gets recorded in the detector, allowing reconstruction of the shower parameters. The available information consists of pulses left by the incoming Cherenkov photons on the photomultiplier tubes, arranged in a plane, the camera. Depending on the energy of the primary gamma, a total of few hundreds to some 10000 Cherenkov photons get collected, in patterns (called the shower image), allowing to discriminate statistically those caused by primary gammas (signal) from the images of hadronic showers initiated by cosmic rays in the upper atmosphere (background). 

Typically, the image of a shower after some pre-processing is an elongated cluster. Its long axis is oriented towards the camera center if the shower axis is parallel to the telescope's optical axis, i.e. if the telescope axis is directed towards a point source. A principal component analysis is performed in the camera plane, which results in a correlation axis and defines an ellipse. If the depositions were distributed as a bivariate Gaussian, this would be an equidensity ellipse. The characteristic parameters of this ellipse (often called Hillas parameters) are among the image parameters that can be used for discrimination. The energy depositions are typically asymmetric along the major axis, and this asymmetry can also be used in discrimination. There are, in addition, further discriminating characteristics, like the extent of the cluster in the image plane, or the total sum of depositions. 

https://archive.ics.uci.edu/ml/datasets/MAGIC+Gamma+Telescope


Attribute Information:

1. fLength: continuous # major axis of ellipse [mm] 
2. fWidth: continuous # minor axis of ellipse [mm] 
3. fSize: continuous # 10-log of sum of content of all pixels [in #phot] 
4. fConc: continuous # ratio of sum of two highest pixels over fSize [ratio] 
5. fConc1: continuous # ratio of highest pixel over fSize [ratio] 
6. fAsym: continuous # distance from highest pixel to center, projected onto major axis [mm] 
7. fM3Long: continuous # 3rd root of third moment along major axis [mm] 
8. fM3Trans: continuous # 3rd root of third moment along minor axis [mm] 
9. fAlpha: continuous # angle of major axis with vector to origin [deg] 
10. fDist: continuous # distance from origin to center of ellipse [mm] 
11. class: g,h # gamma (signal), hadron (background) 

g = gamma (signal): 12332 
h = hadron (background): 6688 

For technical reasons, the number of h events is underestimated. In the real data, the h class represents the majority of the events. 

The simple classification accuracy is not meaningful for this data, since classifying a background event as signal is worse than classifying a signal event as background. For comparison of different classifiers an ROC curve has to be used. The relevant points on this curve are those, where the probability of accepting a background event as signal is below one of the following thresholds: 0.01, 0.02, 0.05, 0.1, 0.2 depending on the required quality of the sample of the accepted events for different experiments.

In [35]:
# Import required libraries
from tpot import TPOTClassifier
from sklearn.cross_validation import train_test_split
import pandas as pd 
import numpy as np

In [36]:
#Load the data
telescope=pd.read_csv('MAGIC Gamma Telescope Data.csv')
telescope.head(5)

,Flength,Fwidth,Fsize,Fconc,Fconc1,Fasym,Fm3long,Fm3trans,Falpha,Fdist,Class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


As can be seen in the above the class object is organized here, and hence for better results, I start with randomly shuffling the data.

In [37]:
telescope_shuffle=telescope.iloc[np.random.permutation(len(telescope))]
telescope_shuffle.head()

,Flength,Fwidth,Fsize,Fconc,Fconc1,Fasym,Fm3long,Fm3trans,Falpha,Fdist,Class
11480,33.9220,19.5203,2.6928,0.2738,0.1450,-12.6419,-21.6683,8.6852,8.8260,152.2470,g
16518,4.2835,3.8870,2.3478,0.8745,0.6158,4.9461,4.5045,-14.1755,21.7552,229.9191,h
8768,100.0110,33.4215,4.0373,0.1147,0.0576,-42.9197,71.0093,-7.2555,1.9529,298.4280,g
14148,129.3620,15.1932,3.2420,0.4023,0.2130,-83.1281,-136.5701,-10.6892,7.4249,304.9583,h
5621,65.7225,23.6661,3.2913,0.1989,0.1030,69.4820,53.4678,11.7673,3.3145,219.8490,g


Above also rearranges the index number and below I basically reset the Index Numbers.

In [38]:
tele=telescope_shuffle.reset_index(drop=True)
tele.head()

,Flength,Fwidth,Fsize,Fconc,Fconc1,Fasym,Fm3long,Fm3trans,Falpha,Fdist,Class
0,33.9220,19.5203,2.6928,0.2738,0.1450,-12.6419,-21.6683,8.6852,8.8260,152.2470,g
1,4.2835,3.8870,2.3478,0.8745,0.6158,4.9461,4.5045,-14.1755,21.7552,229.9191,h
2,100.0110,33.4215,4.0373,0.1147,0.0576,-42.9197,71.0093,-7.2555,1.9529,298.4280,g
3,129.3620,15.1932,3.2420,0.4023,0.2130,-83.1281,-136.5701,-10.6892,7.4249,304.9583,h
4,65.7225,23.6661,3.2913,0.1989,0.1030,69.4820,53.4678,11.7673,3.3145,219.8490,g


# Pre-Processing Data

In [39]:
# Check the Data Type
tele.dtypes

Flength     float64
Fwidth      float64
Fsize       float64
Fconc       float64
Fconc1      float64
Fasym       float64
Fm3long     float64
Fm3trans    float64
Falpha      float64
Fdist       float64
Class        object
dtype: object

Class is a Categorical Variable as can be seen from above. The levels in it are:

In [40]:
for cat in ['Class']:
    print("Levels for catgeory '{0}': {1}".format(cat, tele[cat].unique()))   

Levels for catgeory 'Class': ['g' 'h']


Next, the categorical variable is numerically encoded since TPOT for now cannot handle categorical variables. 'g' is coded as 0 and 'h' as 1.

In [41]:
tele['Class']=tele['Class'].map({'g':0,'h':1})

A check is performed to see if there are any missing values and code then accordingly.

In [42]:
tele = tele.fillna(-999)
pd.isnull(tele).any()

Flength     False
Fwidth      False
Fsize       False
Fconc       False
Fconc1      False
Fasym       False
Fm3long     False
Fm3trans    False
Falpha      False
Fdist       False
Class       False
dtype: bool

In [43]:
tele.shape

(19020, 11)

Finally we store the class labels, which we need to predict, in a separate variable.

In [44]:
tele_class = tele['Class'].values

# Data Analysis using TPOT

To begin our analysis, we need to divide our training data into training and validation sets. The validation set is just to give us an idea of the test set error.

In [45]:
training_indices, validation_indices = training_indices, testing_indices = train_test_split(tele.index, stratify = tele_class, train_size=0.75, test_size=0.25)
training_indices.size, validation_indices.size

(14265, 4755)

After that, we proceed to calling the fit, score and export functions on our training dataset. An important TPOT parameter to set is the number of generations. Here we have set it to 5. On a standard laptop with 4GB RAM, it roughly takes 5 minutes per generation to run. For each added generation, it should take 5 mins more. Thus, for the default value of 100, total run time could be roughly around 8 hours.

In [ ]:
tpot = TPOTClassifier(generations=5, verbosity=2)
tpot.fit(tele.drop('Class',axis=1).loc[training_indices].values, tele.loc[training_indices,'Class'].values)

Optimization Progress:   1%|▏         | 8/600 [00:28<17:54,  1.82s/pipeline]

In the above, 5 generations were computed, each giving the training efficiency of fitting model on the training set. As evident, the best pipeline is the one that has the CV score of 84.573%. The model that produces this result is one that fits extra trees on gradient boosting algorithm. Next, the testing error is computed for validation.

In [ ]:
tpot.score(tele.drop('Class',axis=1).loc[validation_indices].values, tele.loc[validation_indices, 'Class'].values)

As can be seen, the testing accuracy is 84.428%.

In [ ]:
tpot.export('tpot_MAGIC_Gamma_Telescope_pipeline.py')

Below is the code that got generated. As can be seen gradient boosting did the best here. 

In [ ]:
# %load tpot_MAGIC_Gamma_Telescope_pipeline.py
import numpy as np
import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

# NOTE: Make sure that the class is labeled 'class' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', delimiter='COLUMN_SEPARATOR')
training_indices, testing_indices = train_test_split(tpot_data.index, stratify = tpot_data['class'].values, train_size=0.75, test_size=0.25)

result1 = tpot_data.copy()

# Perform classification with a gradient boosting classifier
gbc1 = GradientBoostingClassifier(learning_rate=0.49, max_features=1.0, min_weight_fraction_leaf=0.09, n_estimators=500, random_state=42)
gbc1.fit(result1.loc[training_indices].drop('class', axis=1).values, result1.loc[training_indices, 'class'].values)

result1['gbc1-classification'] = gbc1.predict(result1.drop('class', axis=1).values)
